# Import Statements

In [1]:
import sys
sys.path.append("../../mushu")  # driver for the amps
sys.path.append("../../mushu/libmushu") 
sys.path.append("../../callpyff")   # talk to the stimuli
sys.path.append("../../nftools")  # handy stuff needed for NF
sys.path.append("../../wyrm")  # real-time data analysis

import time
import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
import easygui
import mne
import re
import pickle
import dynarray
import random
import libmushu

from IPython.display import clear_output
from callpyff import bcinetwork, bcixml 
from scipy import signal
import logging  # set log level
logging.basicConfig(level=logging.WARNING)

from nftools.loopcontrol import LoopState
from nftools.analysis import convert_alld_allm_to_mne
from nftools.analysis import select_part_from_mne_dataset
from nftools.analysis import plot_compare_two_spectra  
from nftools import camera

from wyrm.types import RingBuffer
from wyrm.types import BlockBuffer
from wyrm import io
from wyrm import processing as proc

from wyrm import signaltracking
from wyrm import filtering
from wyrm.types import RingBuffer

from numpy.linalg import multi_dot
from sklearn.decomposition import FastICA
from collections import deque

import importlib
import warnings
warnings.filterwarnings('ignore')

import ipdb

# Memory Monitoring Function

In [2]:
import os, psutil
def monitor_memory():
    pid = os.getpid()
    ps = psutil.Process(pid)
    memoryUse = ps.memory_info()
    return memoryUse.rss/1e6

# Define and Configure Amplifier

In [3]:
amp = libmushu.get_amp('bpamp')

In [4]:
# Configure the amp
amp.configure(remotecontrol=True,
              recorderip='20.100.0.3',
              recorderport=51244,
              pathtoworkspace='C:\\Vision\\Workfiles\\NF_64chEEG.rwksp',
              experimentnumber='Pre-Run01',
              subjectid='0001',
              n_channels=64,
              fs=5000)

connecting to 20.100.0.3 port 6700
sending b'1C:\\Vision\\Workfiles\\NF_64chEEG.rwksp'
sending b'2Pre-Run01'
sending b'30001'
sending b'4'
sending b'M'


# Set up BCI Network

In [5]:
bcinet = bcinetwork.BciNetwork('20.100.0.2', bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

In [6]:
feedbacks = bcinet.getAvailableFeedbacks()
print(feedbacks)

['TestD2', 'MovingRhomb', 'LibetClock', 'BrainWaveTraining_II', 'TobiQLAdapter', 'Lesson04', 'EyetrackerRawdata', 'EyetrackerFeedback', 'HexoSpeller', 'P300_Rectangle', 'ERPHex', 'BrainWaveTraining', 'StopVigilanceTask', 'FeedbackCursorArrow', 'TrivialPong', 'CheckerboardVEP', 'HexoSpellerVE', 'BoringClock', 'nback_verbal', 'Lesson01', 'BrainPong', 'CakeSpellerVE', 'MovingRhombGL', 'RestingState', 'NFBasicThermometer', 'RSVPSpeller', 'CenterSpellerVE', 'MultiVisualOddball', 'Lesson01b', 'GoalKeeper', 'EEGfMRILocalizer', 'Oddball', 'StroopFeedback', 'ERPMatrix', 'VisualOddballVE', 'Lesson05', 'Lesson06', 'VisualOddball', 'Lesson02', 'Lesson03']


# Main NF/RS Function

In [7]:
def nf_trial(task, thr=None, count=0):
    %matplotlib qt5  
    plt.ion()  # enable widget plots & interactive plots
    data_length = np.linspace(0,600*5000,num=600*5000)
    fs=5000.
    time_in_plot=6
    sy1=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))
    sy2=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))
    sy3=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))
    sy4=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))
    channel_to_plot=0
    sx=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))
    if task==1:
        bcinet.send_init('BrainWaveTraining_II')
    elif task==0:
        bcinet.send_init('RestingState')
        bcinet.send_signal(bcixml.BciSignal({'EYESCLOSED_TIME': 300.0},None, bcixml.INTERACTION_SIGNAL))
    #c=camera.DoCamera(); c.start()
    bcinet.send_signal(bcixml.BciSignal({'EX_TESTNFNOISE': False},None, bcixml.INTERACTION_SIGNAL))
    
    # if on wrong screen:
    bcinet.send_signal(bcixml.BciSignal({'MONITOR_DISPLAYONSCREEN': 1},None, bcixml.INTERACTION_SIGNAL))
    
    bcinet.play()
    bcinet.send_signal(bcixml.BciSignal({'emgThrContainer': 0.1}, None, bcixml.CONTROL_SIGNAL))
    amp.start()
    
    importlib.reload(signaltracking)

    # from wyrm import signaltracking
    eegfilter = filtering.bwBPF(12,15)
    emgfilter = filtering.bwHPF(50)
    # smoother = filtering.bwExponentialSmoother(0.998)
    # smoother = filtering.bwLPF(2,order=2)
    smoother_eeg = filtering.bwSimpleAverager(300,2)
    smoother_emg = filtering.bwSimpleAverager(300,2)

    track_for_eeg_stimuli = signaltracking.sending_to_nfstim(thr=1.0, dur=0.20, feedback_type='eeg', max4audio=1.2, bcinet=bcinet, st_scaling=10)
    track_for_emg_stimuli = signaltracking.sending_to_nfstim(thr=50, dur=0.15, feedback_type='emg', bcinet=bcinet, st_scaling=350)

    l=LoopState(); l.start()
    alld=dynarray.DynamicArray((None, 2)) 
    alld_ica=dynarray.DynamicArray((None, len(amp.get_channels()))) # data
    allm=[]     # markers
    dts,sent=[],[]
    markeroffset = 0  # needed to store all data in one big mat/vector
    t0=time.time()
    curTime=time.time()
    totalTime = 500
    st=''
    
    #fig=plt.figure(figsize=(20,12))  # plotting...
    #th=fig.suptitle('', fontsize=16)
    #ah1=fig.add_subplot(211)
    #ah2=fig.add_subplot(212)
    
    #l1, = ah1.plot(sx, sy1, color='b', label='Raw')
    #l2, = ah1.plot(sx, sy2, color='r', label='Corrected')
    #l3, = ah2.plot(sx, sy3, color='b', label='Raw')
    #l4, = ah2.plot(sx, sy4, color='r', label='Corrected')
    
    x_data=0
    while l.get_state() != 'Stop' or curTime < totalTime: 
   
        # keep track of time:
        curTime = time.time()
        sfreq = 5000
        ch_names = amp.get_channels()
        markTime=time.time()
        updateTime = 0.01
        rb = RingBuffer(1.0 * 1000)
        # this is where you get the data
        data, marker, annotations = amp.get_data()
        #ipdb.set_trace()
    
        if data.shape[0] > 0:  # this is crucual for remembering filter state.
        
            cnt = io.convert_mushu_data(data, marker, sfreq, ch_names)
            ch_names_copy = ch_names.copy()
            
            
            if count == 0:
                cnt_removed = proc.remove_channels(cnt, ['ECG'])
                cnt_corr = multi_dot([pca_comps.T, M, S, M_inv, pca_comps, cnt_removed.data.T]).T
                #ipdb.set_trace()
                # trouble shooting
                test = proc.select_channels(cnt, ['C3','C4'])
                ch_names_copy.pop(31)
                # ipdb.set_trace()
                cnt_corrected = io.convert_mushu_data(cnt_corr, marker, sfreq, ch_names_copy)
                #ipdb.set_trace()
                eeg_cnt = proc.select_channels(cnt_corrected, ['C3','C4'])
                #ipdb.set_trace()
            elif count == 1:
                eeg_cnt = proc.select_channels(cnt, ['C3','C4'])
                #ipdb.set_trace()
                
            # preprocessing stream for EEG
            # ipdb.set_trace()
            f_eeg_cnt = eegfilter.apply(eeg_cnt)  # so filter it according to specified above
            af_eeg_cnt = proc.absolute(f_eeg_cnt)
            saf_eeg_cnt = smoother_eeg.apply(af_eeg_cnt)
        
            dts.append(saf_eeg_cnt.data.shape[0])
            
        
            # communications with the st-stim computer
            tf, audioTF = track_for_eeg_stimuli.check_above_threshold(saf_eeg_cnt)  # sends markers (should be fast)
            s = track_for_eeg_stimuli.send_data_signal(saf_eeg_cnt) # sends the signal (should also be fast!)
            sent.append(s)

            # do the EMG:
            emg_cnt = proc.select_channels(cnt, ['O1','O2'])
            f_emg_cnt = emgfilter.apply(emg_cnt)  # so filter it according to specified above
            af_emg_cnt = proc.absolute(f_emg_cnt)
            saf_emg_cnt = smoother_emg.apply(af_emg_cnt)
        
            tfemg, audioTFemg = track_for_emg_stimuli.check_above_threshold(saf_emg_cnt)  # sends markers (should be fast)
            semg = track_for_emg_stimuli.send_data_signal(saf_emg_cnt) # sends the signal (should also be fast!)
            
            #if count==0:
            
                #eb_data = cnt.data[:,channel_to_plot]
                #ch_c3 = cnt.data[:, 4]
                #cnt_removed = proc.remove_channels(cnt, ['ECG'])
                
                #data_corrected = multi_dot([pca_comps.T, M, S, M_inv, pca_comps, cnt_removed.data.T])
        
                #sy1.extend(eb_data) 
                #sy2.extend(data_corrected[channel_to_plot,:]*1e-15)
                #sy3.extend(ch_c3)
                #sy4.extend(data_corrected[channel_to_plot,:]*1e-16)
                
                #sx.extend(data_length[x_data:x_data+len(eb_data)]/fs)
        
                #l1.set_ydata(sy1)  
                #l2.set_ydata(sy2)
                #l3.set_ydata(sy3)
                #l4.set_ydata(sy4)
            
                #l1.set_xdata(sx)
                #l2.set_xdata(sx)
                #l3.set_xdata(sx)
                #l4.set_xdata(sx)
        
                #msy1=np.mean(sy1)
                #msy2=np.mean(sy2)
                #msy3=np.mean(sy3)
                #msy4=np.mean(sy4)
         
                #ah1.set_ylim(-150+msy1, 200+msy1)
                #ah1.set_xlim(min(sx), max(sx))
                #ah2.set_ylim(-150+msy3, 200+msy3)
                #ah2.set_xlim(min(sx), max(sx))
        
                #x_data += len(eb_data)
        
                #fig.canvas.draw()
                #fig.canvas.flush_events()
            
            
            alld.extend(eeg_cnt.data)
            alld_ica.extend(data)
            
            for m in marker:
                allm.append([m[0] + markeroffset, m[1]])
            markeroffset += cnt.data.shape[0] / float(sfreq) * 1000.
        
            # append to ringbuffer, so we can calculate features later on on the last N secs/samples of data.
            rb.append(f_eeg_cnt)

            # do the following every 0.1 msec - with with the ringbuffer:
            if curTime - markTime > updateTime:
                # do Stuff
                markTime = curTime
                print('Data is coming in - time = %f' % (curTime - t0))
                str1 = 'Playing Back - time = %f' % (curTime - t0)
                str2 = 'Length Markers: %d' % len(allm)
                str3 = '%d, %d' % data.shape

                # th.set_text(str1 + '\n' + str2 + '\n' +str3)
    amp.stop()   
    
    return alld_ica, ch_names, sfreq

# Obtain first resting-state recording

In [8]:
alld_ica, ch_names, sfreq = nf_trial(0, count=1)

Number of channels: 64
Sampling interval: 200.0
Resolutions: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 10.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Channel Names: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz', 'ECG', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FT9', 'FT10', 'Fpz', 'CPz']
hdr SamplingInterval = 200.00
bwBPF: {'f_low': 12, 'f_high': 15, 'order': 3}
bwHPF: {'f': 50, 'order': 3}
bwSimpleAverager: {'taps': 300, 'f': 2}
bwSimpleAverager: {'taps': 300, 'f'

# Perform ICA

In [9]:
#alld_ica[:,:] /= 1e6
alld_ica.shrink_to_fit()

raw = convert_alld_allm_to_mne(alld_ica, [], ch_names, sfreq)  # covert to MNE

raw.drop_channels(['ECG'])
raw.resample(1000, npad="auto")
#raw.set_eeg_reference('average', projection=True)
#raw.apply_proj()
raw.filter(1., 35., n_jobs=1, fir_design='firwin')
raw.load_data()

ica = mne.preprocessing.ICA(method="infomax", random_state=1)
ica.fit(raw)
ica.plot_sources(inst=raw)
ica.plot_components(inst=raw)
ica_comps = ica.get_sources(inst=raw).get_data()
 
blks = raw.get_data().T
pca_comps = ica.pca_components_
M = ica.mixing_matrix_
M_inv = ica.unmixing_matrix_
S = np.identity(M.shape[0])

The following EEG sensors did not have a position specified in the selected montage: ['ECG']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=64, n_times=197200
    Range : 0 ... 197199 =      0.000 ...    39.440 secs
Ready.
5000.0
Creating RawArray with float64 data, n_channels=1, n_times=197200
    Range : 0 ... 197199 =      0.000 ...    39.440 secs
Ready.
> /home/nfcontrol/nf/nf-rtime/nftools/nftools/analysis.py(148)convert_alld_allm_to_mne()
    147     # create the marker matrix:
--> 148     if len(allm)>0:
    149         raw.add_events(ev_arr)



ipdb>  c


Setting up band-pass filter from 1 - 35 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 8.8 Hz
Filter length of 3301 samples (3.301 sec) selected
Fitting ICA to data using 63 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 63
 
Fitting ICA took 24.5s.


# Define Selection Matrix

In [10]:
S[[3],:] = 0

# Correct Data for Artefacts

In [11]:
blks = np.delete(blks, -1, 1).T # removing STIM channel
data_corr = multi_dot([pca_comps.T, M, S, M_inv, pca_comps, blks])
np.save('data_corr.npy', data_corr)

In [12]:
%matplotlib qt
plt.plot(blks[0,:], 'r'); plt.plot(data_corr[0,:], 'b')

# Thresholding Function

In [13]:
def thresholding(smr_amp):
    stds = [0.5, 1, 1.5, 2]
    interp_fac = 100
    smr_amp = smr_amp[:30000] # take first 30secs of SMR hilbert envelope
            
    mean_y1 = np.mean(smr_amp)
    std_y1 = np.std(smr_amp)
    
    xvals = np.linspace(0,len(smr_amp),len(smr_amp)*interp_fac)
    x = np.arange(0, len(smr_amp))
    smr_interp = np.interp(xvals, x, smr_amp)
    
    num_succ_exc = []
    for a in range(len(stds)):   
        h = np.ones((len(smr_interp),))*(mean_y1+stds[a]*std_y1)
        idx = np.argwhere(np.diff(np.sign(h - smr_interp))).flatten() # where horizontal line intersects with y1
        
        if smr_interp[0] > h[0]: 
            idx = np.delete(idx, 0)
        if smr_interp[-1] > h[0]:
            idx = np.delete(idx, -1)
    
        even, odd = [], []
        for number in range(len(idx)):
            if (number % 2) == 0: 
                even.append(number)
            else:
                odd.append(number) 
        print([len(even), len(idx)])
        
        lengths = []
        print ([len(idx), len(odd), len(even)])
        for b in range(len(even)):
            exc_length = idx[odd[b]]-idx[even[b]]
            lengths.append(exc_length) # calculate distance between intersections (i.e. time of excursion)
        
        num_exc = sum(x >= 200*interp_fac for x in lengths)
        print(num_exc)
        num_succ_exc.append(num_exc)
        best_std_idx = num_succ_exc.index(min(num_succ_exc, key=lambda x:abs(x-9)))
        best_std = stds[best_std_idx]
        
    thr = mean_y1+stds[a]*std_y1
        
    return best_std, thr

In [14]:
c3 = 4 # on Brain Products system only!
best_std, thres = thresholding(data_corr[c3,:])

[3, 6]
[6, 3, 3]
0
[2, 4]
[4, 2, 2]
0
[2, 4]
[4, 2, 2]
0
[1, 2]
[2, 1, 1]
0


In [15]:
# Freeing up memory
del alld_ica, blks, data_corr, raw

# SMR-NF & RS Blocks

## SMR-NF #1

In [16]:
nf_trial(1, thres)

Number of channels: 64
Sampling interval: 200.0
Resolutions: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 10.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Channel Names: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz', 'ECG', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FT9', 'FT10', 'Fpz', 'CPz']
hdr SamplingInterval = 200.00
bwBPF: {'f_low': 12, 'f_high': 15, 'order': 3}
bwHPF: {'f': 50, 'order': 3}
bwSimpleAverager: {'taps': 300, 'f': 2}
bwSimpleAverager: {'taps': 300, 'f'

(DynamicArray(size=1030200, capacity=1843200)([[1375.5, 1228.5, 1174.5, ..., 1199. ,  871.5, 1206.5],
        [1338. , 1212.5, 1161.5, ..., 1181. ,  848. , 1185.5],
        [1300. , 1197.5, 1148.5, ..., 1164.5,  824.5, 1165. ],
        ...,
        [  22. ,  -51.5, -130.5, ...,    9.5, -237. ,  -20. ],
        [ -17.5,  -60. , -117.5, ...,    3.5, -273. ,   -6. ],
        [ -57.5,  -67. , -104.5, ...,   -4. , -311.5,    7.5]]),
 ['Fp1',
  'Fp2',
  'F3',
  'F4',
  'C3',
  'C4',
  'P3',
  'P4',
  'O1',
  'O2',
  'F7',
  'F8',
  'T7',
  'T8',
  'P7',
  'P8',
  'Fz',
  'Cz',
  'Pz',
  'Oz',
  'FC1',
  'FC2',
  'CP1',
  'CP2',
  'FC5',
  'FC6',
  'CP5',
  'CP6',
  'TP9',
  'TP10',
  'POz',
  'ECG',
  'F1',
  'F2',
  'C1',
  'C2',
  'P1',
  'P2',
  'AF3',
  'AF4',
  'FC3',
  'FC4',
  'CP3',
  'CP4',
  'PO3',
  'PO4',
  'F5',
  'F6',
  'C5',
  'C6',
  'P5',
  'P6',
  'AF7',
  'AF8',
  'FT7',
  'FT8',
  'TP7',
  'TP8',
  'PO7',
  'PO8',
  'FT9',
  'FT10',
  'Fpz',
  'CPz'],
 5000)

## RS #1

In [16]:
#ipdb.post_mortem(sys.last_traceback)

> /home/nfcontrol/nf/nf-rtime/wyrm/wyrm/types.py(108)__init__()
    107             assert data.ndim == len(axes) == len(names) == len(units)
--> 108             assert [len(a) for a in axes] == list(data.shape)
    109         self.data = data



ipdb>  p


*** SyntaxError: unexpected EOF while parsing


ipdb>  l


    103         """
    104         if data.size == 0:
    105             pass
    106         else:
    107             assert data.ndim == len(axes) == len(names) == len(units)
--> 108             assert [len(a) for a in axes] == list(data.shape)
    109         self.data = data
    110         self.axes = [np.array(i) for i in axes]
    111         self.names = names
    112         self.units = units
    113 



...

*** SyntaxError: invalid syntax


ipdb>  amp.get_channels


<bound method AmpDecorator.get_channels of <libmushu.ampdecorator.AmpDecorator object at 0x7f25f171f438>>


ipdb>  amp


ipdb>  q


In [14]:
nf_trial(0)

Number of channels: 64
Sampling interval: 200.0
Resolutions: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 10.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Channel Names: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz', 'ECG', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FT9', 'FT10', 'Fpz', 'CPz']
hdr SamplingInterval = 200.00
bwBPF: {'f_low': 12, 'f_high': 15, 'order': 3}
bwHPF: {'f': 50, 'order': 3}
bwSimpleAverager: {'taps': 300, 'f': 2}
bwSimpleAverager: {'taps': 300, 'f'

(DynamicArray(size=1282600, capacity=1638400)([[ 500. ,  315. ,  178. , ...,  -10. ,  340.5,  -49. ],
        [ 500. ,  314. ,  167. , ...,  -17.5,  341.5,  -50.5],
        [ 500. ,  311.5,  156. , ...,  -24. ,  341.5,  -52. ],
        ...,
        [ 220. ,   68. , -504. , ...,   -3. ,   84. ,  -51.5],
        [ 221. ,   67. , -514.5, ...,   -9.5,   84. ,  -53.5],
        [ 221.5,   65.5, -526. , ...,  -17. ,   84.5,  -55.5]]),
 ['Fp1',
  'Fp2',
  'F3',
  'F4',
  'C3',
  'C4',
  'P3',
  'P4',
  'O1',
  'O2',
  'F7',
  'F8',
  'T7',
  'T8',
  'P7',
  'P8',
  'Fz',
  'Cz',
  'Pz',
  'Oz',
  'FC1',
  'FC2',
  'CP1',
  'CP2',
  'FC5',
  'FC6',
  'CP5',
  'CP6',
  'TP9',
  'TP10',
  'POz',
  'ECG',
  'F1',
  'F2',
  'C1',
  'C2',
  'P1',
  'P2',
  'AF3',
  'AF4',
  'FC3',
  'FC4',
  'CP3',
  'CP4',
  'PO3',
  'PO4',
  'F5',
  'F6',
  'C5',
  'C6',
  'P5',
  'P6',
  'AF7',
  'AF8',
  'FT7',
  'FT8',
  'TP7',
  'TP8',
  'PO7',
  'PO8',
  'FT9',
  'FT10',
  'Fpz',
  'CPz'],
 5000)

## SMR-NF #2

In [ ]:
nf_trial(1, thres)

Number of channels: 64
Sampling interval: 200.0
Resolutions: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 10.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Channel Names: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz', 'ECG', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FT9', 'FT10', 'Fpz', 'CPz']
hdr SamplingInterval = 200.00
bwBPF: {'f_low': 12, 'f_high': 15, 'order': 3}
bwHPF: {'f': 50, 'order': 3}
bwSimpleAverager: {'taps': 300, 'f': 2}
bwSimpleAverager: {'taps': 300, 'f'

sending signal -- I ! - 294000 - 19365 - True - -1
sending signal! - 19365 False - 1.296472864368273 -- 0.25
sending signal! - 19365 False - 4.99909191772838 -- 0.3
sending signal! - 19365 False - 5.7834398899739865 -- 0.35
sending signal! - 19365 False - 8.244251138641998 -- 0.39999999999999997
sending signal -- I ! - 294000 - 19365 - True - -1
sending signal! - 19365 False - 1.5905604583072972 -- 0.25
sending signal! - 19365 False - 3.083343686120305 -- 0.3
sending signal -- I ! - 294000 - 19365 - True - -1
sending signal! - 19365 False - 1.447640179748346 -- 0.25
sending signal! - 19365 False - 3.544269536872559 -- 0.3
sending signal! - 19365 False - 25082.528712465915 -- 7.7499999999999805
sending signal! - 19365 False - 25411.38670518644 -- 7.79999999999998
sending signal! - 19365 False - 25741.558005591683 -- 7.84999999999998
sending signal -- I ! - 294000 - 19365 - True - -1
sending signal! - 19365 False - 6.535181365919555 -- 0.2
sending signal! - 19365 False - 15.0799138818812

sending signal! - 19366 False - 1047.5081116686758 -- 1.6000000000000008
sending signal! - 19366 False - 1115.004624196499 -- 1.6500000000000008
sending signal! - 19366 False - 1184.75070397188 -- 1.7000000000000008
sending signal! - 19366 False - 1256.9097456977177 -- 1.7500000000000009
sending signal! - 19366 False - 1330.9877352139017 -- 1.800000000000001
sending signal! - 19366 False - 1406.7663822796112 -- 1.850000000000001
sending signal! - 19366 False - 1484.8377404934383 -- 1.900000000000001
sending signal! - 19366 False - 1565.407140721062 -- 1.950000000000001
sending signal! - 19366 False - 1647.2065795050216 -- 2.000000000000001
sending signal! - 19366 False - 1730.5067011337378 -- 2.0500000000000007
sending signal! - 19366 False - 1816.698004056616 -- 2.1000000000000005
sending signal! - 19366 False - 1905.1526126964093 -- 2.1500000000000004
sending signal! - 19366 False - 1995.6806650197693 -- 2.2
sending signal! - 19366 False - 2088.864147350776 -- 2.25
sending signal! - 

sending signal! - 19367 False - 2580.9397110881973 -- 2.499999999999999
sending signal! - 19369 False - 2685.536377861522 -- 2.549999999999999
sending signal! - 19448 False - 2792.2894516244032 -- 2.5999999999999988
sending signal! - 19527 False - 2901.0537235695915 -- 2.6499999999999986
sending signal! - 19615 False - 3011.7826781089766 -- 2.6999999999999984
sending signal! - 19706 False - 3125.0858858549063 -- 2.7499999999999982
FLIPPING/GOING THE THE MIDDLE AGAIN
sending signal! - 19792 False - 3241.0835373583204 -- 2.799999999999998
sending signal! - 19877 False - 3359.29737948991 -- 2.849999999999998
sending signal! - 19966 False - 3479.182177595628 -- 2.8999999999999977
loop is stopped!


## RS #2

In [ ]:
nf_trial(0)

## SMR-NF #3

In [ ]:
nf_trial(1, thres)

## RS #3

In [ ]:
nf_trial(0)

## SMR-NF #4

In [ ]:
nf_trial(1, thres)

## RS #4

In [ ]:
nf_trial(0)

## SMR-NF #5

In [ ]:
nf_trial(1, thres)

## RS #5

In [ ]:
nf_trial(0)

## SMR-NF #6

In [ ]:
nf_trial(1, thres)

In [ ]:
# EMG
# Scale corrected data to V (from uV)
# PyQT Graph
# Don't plot real-time data next time